<a href="https://colab.research.google.com/github/NakkaBhavith/AI/blob/main/AI_Assign6_450.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import libraries**

In [1]:
import numpy as np
import math as mt
import random
import pandas as pd

# **Import Data**

In [3]:
data = pd.read_excel('/content/drive/MyDrive/AI Data/assign4_data.xls')
data.head()

*** No CODEPAGE record, no encoding_override: will use 'ascii'


,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171


# **Outliers Detection**

In [5]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [6]:
sample_outliers_A = detect_outliers_zscore(data['X'])
sample_outliers_B = detect_outliers_zscore(data['Y'])
print("Outliers in A column from Z-scores method: ", sample_outliers_A)
print("Outliers in B column from Z-scores method: ", sample_outliers_B)

Outliers in A column from Z-scores method:  []
Outliers in B column from Z-scores method:  []


# **Outlier Treatment**

In [7]:
from scipy.stats import zscore
zA = np.abs(zscore(data['X']))
zB = np.abs(zscore(data['Y']))
data['ZscoreA'] = zA
data['ZscoreB'] = zB
outliers_A=len(data[data['ZscoreA']>3])
outliers_B=len(data[data['ZscoreB']>3])
data_z = data[data['ZscoreA']<3]
data=data_z.drop(['ZscoreA', 'ZscoreB'], axis=1)
data

,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171
5,7.8,143
6,7.3,217
7,8.0,190
8,7.1,142
9,7.5,190


# **Missing Values Treatment**

In [8]:
print(data.isnull().sum())

X    0
Y    0
dtype: int64


In [9]:
m1=data['X'].mean()
m2=data['Y'].mean()
data['X'].fillna(value=m1, inplace=True)
data['Y'].fillna(value=m2, inplace=True)
data

,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171
5,7.8,143
6,7.3,217
7,8.0,190
8,7.1,142
9,7.5,190


# **Data Split**

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.X, data.Y, test_size = 0.10, random_state = 10)

# **Initialization**

In [11]:
m1=random.uniform(-1,1)
m2=random.uniform(-1,1)
c=random.uniform(-1,1)
eta=0.0001
max_iter=10

# **Gradient Calculation**

In [12]:
def gradient(m1,m2,c,x,y):
  m1_deriv=-1*(y-m1*x*x-m2*x-c)*x*x
  m2_deriv=-1*(y-m1*x*x-m2*x-c)*x
  c_deriv=-1*(y-m1*x*x-m2*x-c)
  return m1_deriv,m2_deriv,c_deriv

# **Training Using Stochastic Gradient Descent Optimizer**

In [13]:
for iter in range(max_iter):
  for sample in range(x_train.shape[0]):
    del_m1,del_m2,del_c=gradient(m1, m2, c, x_train.iloc[sample], y_train.iloc[sample])
    m1=m1-eta*del_m1
    m2=m2-eta*del_m2
    c=c-eta*del_c
print('model parameter m1, m2 and c',m1,m2,c)

model parameter m1, m2 and c 2.365051509207206 0.7485209818521343 -0.5794934483550871


# **Model Prediction on Training Data**

In [14]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m1 * i*i)+(m2 * i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

# **Model Prediction on Testing Data**

In [15]:
y_pred_test = []
for i in x_test:
  y_p_te = (m1 * i*i)+(m2 * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

# **Error Metrics Calculation**

In [16]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 65.02011671250723
Mean square error 4227.615577308063
Mean absolute error 52.117881360784395


In [17]:
#Testing Accuracies
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse)
mse = mt.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse)  
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Mean square error 2807.782283488349
Root mean square error 52.988510863095115
Mean absolute error 37.29153651549214


In [18]:
x=input("Enter value of input")
estimation=m1*float(x)*float(x)+m2*float(x)+c
print("estimated output:",estimation)

Enter value of input7.1
estimated output: 123.95725210193032
